# 🧠 Predicción de Costos Médicos — *Paso 1: Carga y Preparación (Insurance)*

## 📘 Introducción (Code1)

Este bloque carga el dataset **RegresionTarea.csv** (Insurance de Kaggle), audita su estructura, convierte variables categóricas a *dummies* y separa **X** e **y** con `TARGET = "charges"`.

### 🔧 Qué hace el código
- **Lectura y verificación:** `pd.read_csv(DATA_FILE)` y `assert` para asegurar que el archivo existe.  
- **Auditoría inicial:** `df.info()` para ver filas, tipos y faltantes.  
- **Codificación de categóricas:** `pd.get_dummies(df, drop_first=True)` sobre `sex`, `smoker`, `region` para evitar colinealidad.  
- **Separación:** `y = df["charges"]` y `X = df.drop(columns=["charges"])`.  
- **Chequeo de forma/estadísticos:** imprime `Shape` de `X` y media/desviación/rango de `y`.

---

## 🔍 Resultados del Code1 (con tu salida)

### 📊 Estructura original (`df.info()`)
- **Filas:** 1338  
- **Columnas:** 7  
- **Tipos:**  
  - Numéricas: `age (int64)`, `bmi (float64)`, `children (int64)`, `charges (float64)`  
  - Categóricas: `sex (object)`, `smoker (object)`, `region (object)`  
- **Faltantes:** **ninguno** en ninguna columna  
- **Memoria:** ~73.3 KB

### 🔣 Tras *get_dummies* (`drop_first=True`)
- `sex` (2 niveles) → **1 dummy**  
- `smoker` (2 niveles) → **1 dummy**  
- `region` (4 niveles) → **3 dummies**  
- + Numéricas originales (`age`, `bmi`, `children`)  
- **Total features en X:** **8**  
- **Forma de X:** `(1338, 8)` ✅

### 💵 Estadísticos de la variable objetivo `y = charges`
- **Media:** **13,270.4223**  
- **Desviación estándar:** **12,110.0112**  
- **Rango:** **[1,121.8739, 63,770.4280]**

**Lectura rápida:** la dispersión es alta y el rango muy amplio → probable **asimetría positiva** (cola derecha), típica en costos médicos (p. ej., fumadores elevan la cola).

---

## 🧠 Implicancias y siguientes pasos
- Considerar **transformar** el objetivo para modelos lineales: `y_log = log1p(charges)` para estabilizar varianza.  
- Evaluar **MAE** junto con **RMSE** (RMSE es sensible a outliers).  
- EDA recomendado: histogramas/boxplots de `charges`, comparación por `smoker` y `region`, y correlaciones con `age`/`bmi`.  

---


In [4]:
# =========================================
# 1) Cargar datos y objetivo (Insurance dataset)
# =========================================
import os, json, warnings, platform, datetime
import numpy as np
import pandas as pd
import joblib
warnings.filterwarnings("ignore")

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Nombre del archivo descargado de Kaggle
DATA_FILE = "RegresionTarea.csv"     # <-- tu archivo limpio de Kaggle
TARGET    = "charges"           # variable objetivo

assert os.path.exists(DATA_FILE), f"No se encuentra {DATA_FILE}"

# Leer dataset
df = pd.read_csv(DATA_FILE)
df.info()

# Variables categóricas: sex, smoker, region → convertir a dummies (0/1)
df = pd.get_dummies(df, drop_first=True)

# Definir X y y
y  = df[TARGET]
X  = df.drop(columns=[TARGET])

print("Shape:", X.shape,
      "| y(mean):", round(y.mean(), 4),
      "| y(std):", round(y.std(), 4),
      "| y[min,max]:", (round(y.min(), 4), round(y.max(), 4)))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB
Shape: (1338, 8) | y(mean): 13270.4223 | y(std): 12110.0112 | y[min,max]: (1121.8739, 63770.428)


In [5]:
X , y

(      age     bmi  children  sex_male  smoker_yes  region_northwest  \
 0      19  27.900         0     False        True             False   
 1      18  33.770         1      True       False             False   
 2      28  33.000         3      True       False             False   
 3      33  22.705         0      True       False              True   
 4      32  28.880         0      True       False              True   
 ...   ...     ...       ...       ...         ...               ...   
 1333   50  30.970         3      True       False              True   
 1334   18  31.920         0     False       False             False   
 1335   18  36.850         0     False       False             False   
 1336   21  25.800         0     False       False             False   
 1337   61  29.070         0     False        True              True   
 
       region_southeast  region_southwest  
 0                False              True  
 1                 True             False  
 2

# 🔀 Split temprano (80/20) — *Code2*

## 🧩 ¿Qué hace el código?
- **Función:** `train_test_split(X, y, test_size=0.20, random_state=RANDOM_STATE)`
  - Separa el dataset en **entrenamiento (80%)** y **prueba (20%)**.
  - Usa la misma semilla (`RANDOM_STATE=42`) para **reproducibilidad**.
- **Objetivo del split temprano:** fijar un **conjunto de prueba independiente** desde el inicio para evaluar el rendimiento **fuera de muestra** sin sesgos.
- **Nota (regresión):** no se usa `stratify` (aplica a clasificación). En regresión, es buena práctica verificar que la **distribución de `y`** en *train/test* sea similar (ver chequeos abajo).
- **Prevención de *leakage*:** hacer el split **antes** de cualquier ajuste dependiente de los datos (escalado, selección de variables, imputación, tuning, etc.), idealmente encapsulados luego en un **pipeline**.

---

## 📊 Resultado obtenido
- **Train:** `(1070, 8)` → 1,070 filas y 8 *features*  
- **Test:** `(268, 8)` → 268 filas y 8 *features*  
✔️ Las proporciones **80/20** coinciden con lo esperado y los tamaños suman las 1,338 observaciones totales.

### 🧠 Interpretación
- Con ~**1,070** ejemplos para entrenar, hay suficiente datos para un **baseline** con CV y para **tuning** moderado.
- **268** ejemplos en *test* proveen una evaluación **estable** (varianza de métricas razonable).
- Próximo paso recomendado: confirmar que *train* y *test* mantienen estadísticos similares de `y` (media/desv/rango) para evitar un *split* accidentalmente sesgado.

---

In [6]:
# =========================================
# 2) Split temprano (80/20)
# =========================================
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RANDOM_STATE
)

print(f"Train: {X_train.shape} | Test: {X_test.shape}")

Train: (1070, 8) | Test: (268, 8)


# 🧼 Preprocesamiento en Pipeline — *Code3*

## 🧩 ¿Qué hace el código?
- **ColumnTransformer (preprocessor):** aplica `StandardScaler()` **solo** a las columnas numéricas (que aquí incluyen las dummies), con `remainder="drop"` para usar únicamente esas columnas.
- **Por qué todas son numéricas:** ya hiciste `get_dummies`, así que `sex`, `smoker`, `region` quedaron como 0/1.  
- **VarianceThreshold(0.0):** elimina columnas **constantes** (varianza cero). Útil como red de seguridad ante features degeneradas; en este dataset no debería eliminar ninguna.
- **ImbPipeline:** se usa el `Pipeline` de `imblearn` por consistencia de API; **no** se aplica SMOTE (es regresión).  
- **build_pipe(model):** crea un pipeline ordenado → `("prep" → "var0" → "model")` para evitar **leakage**: el escalado se ajusta **solo con train** y luego se aplica a test.

### ✨ Efecto del escalado
- **Beneficia**: modelos sensibles a escala (Regresión Lineal/Ridge/Lasso, SVR, MLP).  
- **Neutro**: árboles/RandomForest/GBDT (no necesitan escalado, pero no molesta).  
- **Dummies 0/1**: escalarlas no daña; quedan centradas y con varianza unitaria junto al resto.

---

## 📊 Resultado del Code3 (tus salidas)
- **Features numéricas (8):**  
  `['age', 'bmi', 'children', 'sex_male', 'smoker_yes', 'region_northwest', 'region_southeast', 'region_southwest']`
- **Features categóricas:** `[]`  
✔️ Consistente con el *one-hot* previo y con el plan de escalar **todas** las columnas usadas por el modelo.

---

In [7]:
# =========================================
# 3) Preprocesamiento (en pipeline)
# =========================================
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from imblearn.pipeline import Pipeline as ImbPipeline  # imblearn solo por consistencia de API

# Como ya hicimos get_dummies, todas son numéricas
cat_features = []
num_features = X_train.columns.tolist()

# Preprocesador: solo escalado
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_features),
    ],
    remainder="drop",
)

def build_pipe(model):
    # Nota: en regresión NO se usa SMOTE
    return ImbPipeline([
        ("prep", preprocessor),
        ("var0", VarianceThreshold(0.0)),  # limpia columnas constantes
        ("model", model),
    ])

print(f"Features numéricas: {num_features}")
print(f"Features categóricas: {cat_features}")

Features numéricas: ['age', 'bmi', 'children', 'sex_male', 'smoker_yes', 'region_northwest', 'region_southeast', 'region_southwest']
Features categóricas: []


In [ ]:
!pip install xgboost lightgbm catboost

# 🤖 Modelos candidatos (Regresión) — *Code4*

## 🧩 ¿Qué define este bloque?
Se arma una **lista de candidatos** `candidates` con modelos de regresión que cubren distintas hipótesis (linealidad, no linealidad, interacciones) y diferentes sesgos/varianzas.  
Cada modelo se entrenará dentro del **pipeline** (`build_pipe`) que ya escala y filtra varianza cero.

**Lineales (baseline y regularizados)**
- **LR — LinearRegression()**: línea base sin regularización; sensible a colinealidad y outliers.
- **RG — Ridge(random_state=42)**: L2; reduce varianza, robusto ante colinealidad.
- **LS — Lasso(max_iter=5000, random_state=42)**: L1; hace selección de variables (coeficientes a 0).
- **EN — ElasticNet(max_iter=5000, random_state=42)**: mezcla L1+L2; balancea selección y estabilidad.

**Basado en instancias**
- **KNR — KNeighborsRegressor()**: no paramétrico; depende de la **escala** (por eso escalamos). Sensible a ruido y dimensión.

**Árboles y *ensembles***
- **DTR — DecisionTreeRegressor(random_state=42)**: no lineal, captura interacciones; alto riesgo de **overfitting** sin poda.
- **RFR — RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)**: promedia muchos árboles → menor varianza; robusto a outliers; poco interpretable globalmente.

**Red neuronal**
- **MLP — MLPRegressor(hidden_layer_sizes=(64,), max_iter=800, random_state=42)**: puede modelar no linealidades; requiere buen escalado y tuning (capas, *learning rate*, regularización).

**Gradient Boosting (árboles potenciados)**
- **XGB — XGBRegressor(..., n_estimators=400, learning_rate=0.05, max_depth=6, subsample=0.9, colsample_bytree=0.9)**  
- **LGB — LGBMRegressor(..., n_estimators=500, learning_rate=0.05, subsample=0.9, colsample_bytree=0.9)**  
- **CAT — CatBoostRegressor(iterations=600, depth=6, learning_rate=0.05, verbose=False)**  
Todos capturan **no linealidad** e **interacciones** de forma eficiente; suelen rendir muy bien en tabulares. Aquí las categóricas ya están en *one-hot*, por lo que CatBoost operará solo con numéricas (también funciona).

In [12]:
# =========================================
# 4) Modelos candidatos (REGRESIÓN)
# =========================================
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

candidates = [
    ("LR",  LinearRegression()),
    ("RG",  Ridge(random_state=RANDOM_STATE)),
    ("LS",  Lasso(random_state=RANDOM_STATE, max_iter=5000)),
    ("EN",  ElasticNet(random_state=RANDOM_STATE, max_iter=5000)),
    ("KNR", KNeighborsRegressor()),
    ("DTR", DecisionTreeRegressor(random_state=RANDOM_STATE)),
    ("RFR", RandomForestRegressor(n_estimators=300, random_state=RANDOM_STATE, n_jobs=-1)),
    ("MLP", MLPRegressor(hidden_layer_sizes=(64,), max_iter=800, random_state=RANDOM_STATE)),
    ("XGB", XGBRegressor(tree_method="hist", random_state=RANDOM_STATE,
                         n_estimators=400, learning_rate=0.05, max_depth=6,
                         subsample=0.9, colsample_bytree=0.9, n_jobs=-1)),
    ("LGB", LGBMRegressor(n_estimators=500, learning_rate=0.05, max_depth=-1,
                          subsample=0.9, colsample_bytree=0.9,
                          random_state=RANDOM_STATE, n_jobs=-1, verbosity=-1)),
    ("CAT", CatBoostRegressor(iterations=600, learning_rate=0.05, depth=6,
                              random_state=RANDOM_STATE, l2_leaf_reg=3.0,
                              verbose=False, allow_writing_files=False, thread_count=-1)),
]

## 📊 Resultados e interpretación

### Ranking (promedios en test CV)
1. **CAT** — RMSE **4809.92**, MAE **2745.85**, R² **0.838** ✅  
2. **RFR** — RMSE 4940.32, MAE 2810.94, R² 0.830  
3. **XGB** — RMSE 5156.30, MAE 3071.67, R² 0.815  
4. **LGB** — RMSE 5171.88, MAE 3205.26, R² 0.814  
5. **KNR** — RMSE 5629.18, MAE 3507.49, R² 0.779  
6–8. **LS / RG / LR** — RMSE ~**6123**, MAE ~**4235**, R² ~**0.739** (línea base lineal/regularizada)  
9. **DTR** — RMSE 6671.85, MAE 3237.99, R² 0.689  
10. **EN** — RMSE 7050.55, MAE 5164.37, R² 0.654  
11. **MLP** — RMSE 15606.54, MAE 11280.46, R² **-0.693** (⚠️ no convergió; ver aviso)

### Lecturas clave
- **Ganador baseline:** **CatBoostRegressor** (mejor RMSE y mejor R²). Los **ensembles de árboles** dominan el problema (CAT ≈ RF ≈ XGB/LGB), lo esperado en datos tabulares con no linealidades e interacciones (p. ej., `smoker_yes × bmi × age`).
- **Lineales (LR/Ridge/Lasso):** rendimiento sólido pero **inferior** a árboles potenciados; capturan solo relaciones principalmente aditivas.
- **KNN:** razonable pero por detrás de ensembles; sensible a escala y a la elección de `n_neighbors` (aún sin tuning).
- **Árbol simple (DTR):** peor que sus versiones en conjunto (**RF/Boosting**) por **alta varianza**.
- **ElasticNet:** debajo de Ridge/Lasso sin tuning; la mezcla L1+L2 no ayudó con hiperparámetros por defecto.
- **MLP:** **ConvergenceWarning** y desempeño muy pobre → requiere **early stopping**, regularización, ajuste de arquitectura y `max_iter` mayor.


In [13]:
# =========================================
# 5) Baseline con CV (sin tuning)
# =========================================
from sklearn.model_selection import KFold, cross_validate
import pandas as pd

cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
scoring = {
    "rmse": "neg_root_mean_squared_error",
    "mae":  "neg_mean_absolute_error",
    "r2":   "r2",
}

rows = []
for name, model in candidates:
    pipe = build_pipe(model)
    scores = cross_validate(pipe, X_train, y_train, cv=cv, scoring=scoring, n_jobs=-1)
    row = {
        "model": name,
        "rmse": -scores["test_rmse"].mean(),
        "mae":  -scores["test_mae"].mean(),
        "r2":    scores["test_r2"].mean(),
    }
    rows.append(row)
    print(f"{name:>3} | RMSE {row['rmse']:.3f} | MAE {row['mae']:.3f} | R² {row['r2']:.3f}")

baseline_df = pd.DataFrame(rows).sort_values("rmse").reset_index(drop=True)
display(baseline_df)

baseline_best_name  = baseline_df.iloc[0]["model"]
baseline_best_model = dict(candidates)[baseline_best_name]
print(f">>> Baseline ganador: {baseline_best_name}")

 LR | RMSE 6123.354 | MAE 4234.984 | R² 0.739
 RG | RMSE 6123.310 | MAE 4236.462 | R² 0.739
 LS | RMSE 6123.308 | MAE 4234.844 | R² 0.739
 EN | RMSE 7050.553 | MAE 5164.369 | R² 0.654
KNR | RMSE 5629.177 | MAE 3507.490 | R² 0.779
DTR | RMSE 6671.849 | MAE 3237.991 | R² 0.689
RFR | RMSE 4940.323 | MAE 2810.938 | R² 0.830


/opt/conda/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptro

MLP | RMSE 15606.537 | MAE 11280.464 | R² -0.693
XGB | RMSE 5156.305 | MAE 3071.671 | R² 0.815
LGB | RMSE 5171.876 | MAE 3205.260 | R² 0.814
CAT | RMSE 4809.922 | MAE 2745.851 | R² 0.838


,model,rmse,mae,r2
0,CAT,4809.921630,2745.851251,0.838345
1,RFR,4940.323260,2810.937874,0.829974
2,XGB,5156.304618,3071.671276,0.814907
3,LGB,5171.875600,3205.260495,0.813986
4,KNR,5629.177182,3507.489693,0.779218
5,LS,6123.307570,4234.844428,0.738858
6,RG,6123.310369,4236.461734,0.738857
7,LR,6123.353823,4234.983570,0.738853
8,DTR,6671.848774,3237.991359,0.688597
9,EN,7050.553194,5164.369141,0.654384


>>> Baseline ganador: CAT


# 🔧 Tuning con CV y elección del ganador — *Code6*

## 🧩 ¿Qué hace el código?
- **Estrategia de búsqueda:** `RandomizedSearchCV` sobre un **pipeline** (`prep → var0 → model`) para evitar *leakage*.
- **Esquemas de CV:**
  - **Modelos “ligeros”** (RG, EN): `KFold(n_splits=5, shuffle=True, random_state=42)`.
  - **Modelos “pesados”** (RFR, XGB, LGB, CAT): `KFold(n_splits=3, ...)` para acelerar.
- **Espacios de hiperparámetros:** distribuciones amplias (p. ej., `loguniform` para `alpha`/`learning_rate`, `randint` para profundidad/árboles/hojas).
- **Parámetros clave del `RandomizedSearchCV`:**
  - `n_iter`: 12 para ligeros, **15** para pesados.
  - `scoring`: **RMSE/MAE/R²**; **`refit="rmse"`** → el mejor se elige por **menor RMSE** en CV.
  - `n_jobs=-1`, `random_state=42`, `error_score=np.nan`.
- **Cache opcional:** `pipe.set_params(memory=cache_dir)` (si el estimador lo soporta) para reutilizar transformaciones y **acelerar**.
- **Salida final:** colecciona `(nombre, mejor_estimator, mejor_RMSE_CV, mejores_params)` y **ordena** por RMSE para declarar el **ganador optimizado**.

---

## 📊 Resultado e interpretación (tus salidas)
**Logs de ajuste:**
- RG (12×5 folds) → 60 *fits*  
- EN (12×5 folds) → 60 *fits*  
- RFR/XGB/LGB/CAT (15×3 folds) → 45 *fits* cada uno

**Ganador optimizado (según RMSE CV):**  
> **RFR — RandomForestRegressor**  
> **RMSE CV = 4559.799** (menor es mejor)  
> **Mejores hiperparámetros:**
> - `bootstrap`: **True**  
> - `max_depth`: **5**  
> - `max_features`: **None**  
> - `min_samples_leaf`: **7**  
> - `min_samples_split`: **13**  
> - `n_estimators`: **463**

### 🧠 Lecturas clave
- **Mejora vs. baseline CV:** el mejor baseline fue **CAT** con RMSE ≈ **4809.92** (Code5, 5-fold). El tuning produjo **RFR 4559.80** (3-fold). **Indicativamente mejora** (~250 pts RMSE), aunque:
  - ⚠️ **Advertencia de comparabilidad:** los **folds** no son idénticos (CAT baseline en 5-fold vs RFR tuning en 3-fold). Aun así, la magnitud de mejora sugiere **progreso real**. Se debe **validar en TEST**.
- **Hiperparámetros aprendidos**:
  - `max_depth=5`, `min_samples_leaf=7`, `min_samples_split=13` → **control de complejidad** para evitar sobreajuste.
  - `n_estimators=463` garantiza estabilidad de la media del bosque.
  - `max_features=None` (toma todas las features por división) puede mejorar ajuste en datasets con pocas columnas (8), compensado por la poda suave (`depth`/`leaf`/`split`).
- **Siguiente paso crítico:** evaluar el **pipeline óptimo** en el **conjunto de test** (Code2) y comparar **RMSE/MAE/R²** con el baseline.


In [15]:
# =========================================
# 6) Tuning con CV y elección del ganador (rápido) para tu dataset
# =========================================
import tempfile, shutil
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
try:
    from scipy.stats import loguniform
except Exception:
    from sklearn.utils.fixes import loguniform

# CV: más fuerte para modelos ligeros, más suave para pesados
cv_light = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
cv_heavy = KFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

# Espacios de hiperparámetros
param_spaces = {
    "RG":  {"model__alpha": loguniform(1e-3, 1e3)},
    "EN":  {"model__alpha": loguniform(1e-3, 1e2), "model__l1_ratio": uniform(0.0, 1.0)},
    "RFR": {"model__n_estimators": randint(200, 600), "model__max_depth": randint(4, 16),
            "model__min_samples_split": randint(2, 20), "model__min_samples_leaf": randint(1, 10),
            "model__max_features": ["sqrt","log2", None], "model__bootstrap": [True, False]},
    "XGB": {"model__n_estimators": randint(250, 600), "model__learning_rate": loguniform(5e-3, 2e-1),
            "model__max_depth": randint(3, 9), "model__subsample": uniform(0.7, 0.3),
            "model__colsample_bytree": uniform(0.7, 0.3), "model__min_child_weight": randint(1, 6)},
    "LGB": {"model__n_estimators": randint(300, 800), "model__learning_rate": loguniform(5e-3, 2e-1),
            "model__num_leaves": randint(16, 128), "model__max_depth": randint(-1, 12),
            "model__min_child_samples": randint(10, 50), "model__subsample": uniform(0.7, 0.3),
            "model__colsample_bytree": uniform(0.7, 0.3), "model__reg_lambda": loguniform(1e-3, 10)},
    "CAT": {"model__iterations": randint(300, 700), "model__learning_rate": loguniform(5e-3, 2e-1),
            "model__depth": randint(4, 10), "model__l2_leaf_reg": loguniform(1e-2, 30),
            "model__border_count": randint(32, 255)},
}

# Modelos a optimizar
to_tune = [
    ("RG",  Ridge(random_state=RANDOM_STATE)),
    ("EN",  ElasticNet(random_state=RANDOM_STATE, max_iter=5000)),
    ("RFR", RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=1)),
    ("XGB", XGBRegressor(tree_method="hist", random_state=RANDOM_STATE, n_jobs=1)),
    ("LGB", LGBMRegressor(random_state=RANDOM_STATE, n_jobs=1, verbosity=-1)),
    ("CAT", CatBoostRegressor(random_state=RANDOM_STATE, verbose=False, allow_writing_files=False, thread_count=1)),
]

# Métricas de evaluación
refit_metric = "rmse"  # minimizar RMSE
scoring = {"rmse": "neg_root_mean_squared_error", 
           "mae": "neg_mean_absolute_error", 
           "r2": "r2"}

# Lista de resultados
best_models = []
cache_dir = tempfile.mkdtemp(prefix="skcache_")

try:
    for name, base_model in to_tune:
        pipe = build_pipe(base_model)
        try:
            pipe.set_params(memory=cache_dir)
        except:
            pass

        heavy = name in ["RFR","XGB","LGB","CAT"]

        search = RandomizedSearchCV(
            pipe,
            param_spaces[name],
            n_iter=(15 if heavy else 12),
            cv=(cv_heavy if heavy else cv_light),
            scoring=scoring,
            refit="rmse",
            n_jobs=-1,
            random_state=RANDOM_STATE,
            verbose=1,
            error_score=np.nan,
            return_train_score=False
        )

        search.fit(X_train, y_train)
        best_models.append((name, search.best_estimator_, -search.best_score_, search.best_params_))  # RMSE positivo

    # Ordenar por menor RMSE
    best_models.sort(key=lambda x: x[2])
    best_name, final_pipe_opt, best_cv_rmse, best_params = best_models[0]

    print(f">>> GANADOR OPTIMIZADO: {best_name} (RMSE CV={best_cv_rmse:.3f})")
    print("Mejores hiperparámetros encontrados:", best_params)

finally:
    shutil.rmtree(cache_dir, ignore_errors=True)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 3 folds for each of 15 candidates, totalling 45 fits
Fitting 3 folds for each of 15 candidates, totalling 45 fits
Fitting 3 folds for each of 15 candidates, totalling 45 fits
Fitting 3 folds for each of 15 candidates, totalling 45 fits
>>> GANADOR OPTIMIZADO: RFR (RMSE CV=4559.799)
Mejores hiperparámetros encontrados: {'model__bootstrap': True, 'model__max_depth': 5, 'model__max_features': None, 'model__min_samples_leaf': 7, 'model__min_samples_split': 13, 'model__n_estimators': 463}


# ⚖️ Comparación justa (solo CV) — *Code7*

## 🧩 ¿Qué hace el código?
- **Mismo esquema de CV para ambos**: `same_cv = KFold(n_splits=5, shuffle=True, random_state=123)`  
  → garantiza una **comparación manzana con manzana** (mismos folds para baseline y tuned).
- **Pipelines comparados**:
  - `pipe_baseline_best`: pipeline con el **mejor baseline** de Code5 (`baseline_best_model`).
  - `pipe_tuned_best`  : pipeline **optimizado** de Code6 (`final_pipe_opt`).
- **Métrica usada**: **RMSE** (menor es mejor) computada con `cross_validate` en los **mismos folds**.
- **Regla de decisión**:
  - Si la mejora relativa \((\text{RMSE}_{base} - \text{RMSE}_{tuned}) / \text{RMSE}_{base} \ge 1\%\) → **gana el tuned**.
  - Si no, **nos quedamos con el baseline** por **simplicidad** y menor riesgo.
- **Salida esperada**:
  - Imprime dos líneas tipo:  
    `Baseline(CAT): RMSE 4xxx.xxxx`  
    `Tuned(RFR): RMSE 4xxx.xxxx`  
  - Luego: `>>> Modelo seleccionado para TEST: <nombre>`

---

## 🔍 Cómo interpretar la salida
- **Caso A — Tuned mejora ≥ 1%**  
  - *Ej.* `Baseline(CAT): RMSE 4810` vs `Tuned(RFR): RMSE 4560`  
  - **Conclusión**: el **tuned** ofrece **ganancia real** y consistente en los mismos folds → **avanzar con tuned** a evaluación en **TEST**.
- **Caso B — Mejora < 1%**  
  - *Ej.* `Baseline(CAT): RMSE 4810` vs `Tuned(RFR): RMSE 4780`  
  - **Conclusión**: la diferencia es marginal; por **parquedad** y **robustez**, quedarse con el **baseline**.
- **Buenas prácticas**:
  - Reporta el **% de mejora**: \(\Delta\% = 100 \times (\text{RMSE}_{base} - \text{RMSE}_{tuned}) / \text{RMSE}_{base}\).
  - Si dudas por **varianza**, repite con otro `random_state` o usa **repeated KFold** para estimar la **incertidumbre** de la mejora.
  - El ganador de esta sección es el que se usará en el **TEST hold-out** (Code2) en el siguiente paso.


In [26]:
# =========================================
# 7) Comparación justa (solo CV) - baseline vs ganador
# =========================================
from sklearn.model_selection import KFold, cross_validate

same_cv = KFold(n_splits=5, shuffle=True, random_state=123)
pipe_baseline_best = build_pipe(baseline_best_model)
pipe_tuned_best    = final_pipe_opt

def cv_rmse(pipe, name):
    s = cross_validate(pipe, X_train, y_train, cv=same_cv,
                       scoring={"rmse":"neg_root_mean_squared_error"}, n_jobs=-1)
    rmse = -s["test_rmse"].mean()
    print(f"{name}: RMSE {rmse:.4f}")
    return rmse

rmse_base = cv_rmse(pipe_baseline_best, f"Baseline({baseline_best_name})")
rmse_tune = cv_rmse(pipe_tuned_best,   f"Tuned({best_name})")

# Regla: si la mejora < 1% del RMSE base, nos quedamos con el baseline (más simple)
if (rmse_base - rmse_tune) / rmse_base >= 0.01:
    winner_name, winner_pipe = best_name, pipe_tuned_best
else:
    winner_name, winner_pipe = baseline_best_name, pipe_baseline_best

print(f">>> Modelo seleccionado para TEST: {winner_name}")


Baseline(CAT): RMSE 4832.2032
Tuned(RFR): RMSE 4571.8756
>>> Modelo seleccionado para TEST: RFR


# 🛡️ Política de decisión (postprocesado) — *Code8*

## 🧩 ¿Qué hace el código?
- **Define una política** (`POLICY`) para ajustar las predicciones antes de evaluarlas/desplegarlas:
  - `clip_to_train_range=True` → **recorta** predicciones al **rango observado en TRAIN**.
  - `round_to_int=False` → no redondea (apropiado para `charges`, variable continua).
  - `lower` / `upper` → límites tomados de `y_train.min()` y `y_train.max()`.
- **Función `postprocess_preds`**:
  - Copia `yhat`.
  - Aplica `np.clip(yhat, lower, upper)` si el recorte está activo.
  - Redondea a entero solo si `round_to_int=True`.

---

## 🔍 Cómo interpretar la salida
- **Propósito del recorte:** evitar **extrapolaciones extremas** (negativas o fuera de escala) y estabilizar el comportamiento del modelo en colas.
- **Límites basados en TRAIN:** previene *leakage* (no usa información de TEST).  
- **Riesgo:** si en producción hay valores reales fuera del rango de TRAIN, el recorte puede introducir **sesgo** (sub/sobreestimación en extremos).
- **Redondeo:** mantener **`False`** para `charges`; activar solo si el objetivo es **discreto** (conteos).

**Chequeo recomendado (opcional):** calcular **% de predicciones recortadas** y comparar métricas **antes vs. después** del postprocesado para verificar su impacto.


In [27]:
# =========================================
# 8) Política de decisión (mínima)
# =========================================
POLICY = {
    "clip_to_train_range": True,   # recorta predicciones al rango visto en TRAIN
    "round_to_int": False,         # pon True si el objetivo es entero (conteos)
    "lower": float(y_train.min()),
    "upper": float(y_train.max()),
}
print("Política:", POLICY)

def postprocess_preds(yhat, policy=POLICY):
    ypp = yhat.copy()
    if policy.get("clip_to_train_range", False):
        ypp = np.clip(ypp, policy["lower"], policy["upper"])
    if policy.get("round_to_int", False):
        ypp = np.rint(ypp).astype(int)
    return ypp


Política: {'clip_to_train_range': True, 'round_to_int': False, 'lower': 1121.8739, 'upper': 62592.87309}


# 🧪 Evaluación final en TEST — *Code9*

## 🧩 ¿Qué hace el código?
- **Entrena** el `winner_pipe` con `X_train, y_train`.
- **Predice** en `X_test` y aplica el **postprocesado** (`postprocess_preds`) según la **POLICY** (recorte al rango de TRAIN).
- **Calcula métricas** en TEST:
  - **RMSE** (raíz del error cuadrático medio) — penaliza más los errores grandes.
  - **MAE** (error absoluto medio) — interpretable como error promedio en unidades de `charges`.
  - **R²** — proporción de varianza explicada.
- **Muestra** un **vistazo de 10 casos** (`y_true` vs `y_pred`) para inspección rápida.

---
## 🔍 Interpretación
- **Desempeño global**:
  - **R² = 0.8787** → el modelo explica ~**87.9%** de la varianza de `charges` en **datos no vistos**.
  - **RMSE ≈ 4,339** y **MAE ≈ 2,471**. Con media de `charges` ≈ **13,270** (Code1), esto implica:
    - `RMSE / media ≈ 33%` → error típico cuadrático en torno a un tercio de la media.
    - `MAE / media ≈ 19%` → error absoluto promedio cercano a una quinta parte de la media.
- **Consistencia vs CV**:
  - El tuning reportó **RMSE CV ≈ 4,560** (Code6). En TEST obtienes **4,339**, **ligeramente mejor** (posible variación de muestra; señal de **generalización razonable**).
- **Patrón en ejemplos**:
  - Hay **sobreestimaciones** moderadas (p. ej., 9,095 → 10,253; +1,157) y **subestimaciones** en casos altos (p. ej., 29,331 ← 27,553; −1,778), coherentes con el compromiso sesgo-varianza del bosque.

---

In [28]:
# =========================================
# 9) Evaluación final en TEST
# =========================================
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

winner_pipe.fit(X_train, y_train)
y_pred = winner_pipe.predict(X_test)
y_pp   = postprocess_preds(y_pred, POLICY)

rmse = mean_squared_error(y_test, y_pp, squared=False)
mae  = mean_absolute_error(y_test, y_pp)
r2   = r2_score(y_test, y_pp)

print(f"TEST → RMSE: {rmse:.4f} | MAE: {mae:.4f} | R²: {r2:.4f}")

# vistazo rápido (primeros 10)
import pandas as pd
preview = pd.DataFrame({"y_true": y_test.reset_index(drop=True),
                        "y_pred": pd.Series(y_pp)}).head(10)
print(preview.to_string(index=False))


TEST → RMSE: 4339.3197 | MAE: 2470.5496 | R²: 0.8787
     y_true       y_pred
 9095.06825 10252.584961
 5272.17580  5932.748470
29330.98315 27553.068281
 9301.89355 10602.135929
33750.29180 34827.732703
 4536.25900  6569.321544
 2117.33885  2345.542471
14210.53595 14145.541554
 3732.62510  5355.007769
10264.44210 11524.080489


# 🔎 Interpretabilidad + Análisis de Error (mínimo) — *Code10*

## 🧩 ¿Qué hace el código?
- **Recorte aplicado por la política:** calcula el % de predicciones que quedarían fuera del rango de TRAIN y serían **recortadas**.
- **Importancia por permutación (pipeline completo):** mide cuánto **empeora el RMSE** cuando se desordena cada *feature* original, evaluando la importancia **en el flujo real** (prep + modelo).
- **Análisis de errores:** construye `|error|` por fila, resume su **distribución** y lista los **10 peores casos** con sus *features*.
- **Subgrupos (opcional):** si existe `clase_salario`, reporta **MAE por grupo**.

---
## 🔍 Interpretación
- **Factor crítico:** *smoker_yes* es **clavísimo** (con diferencia) en la predicción de `charges`.  
- **No linealidades/colas:** los grandes errores se concentran en **altos costos médicos** (cola derecha). Esto es típico; incluso los ensembles pueden **subestimar** extremos.
- **Variables con poco aporte:** regiones y sexo apenas mueven la aguja en este conjunto (posible señal de que su efecto está absorbido por las otras variables).

---

In [29]:
# =========================================
# 10) Interpretabilidad + breve error analysis (mínimo, FIX)
# =========================================
import numpy as np
import pandas as pd
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error

# 10.1 ¿Cuánto recorta la política?
raw_pred = winner_pipe.predict(X_test)
clip_low  = (raw_pred < POLICY["lower"]).mean()
clip_high = (raw_pred > POLICY["upper"]).mean()
print(f"[Policy] clipped_low: {clip_low:.3%} | clipped_high: {clip_high:.3%}")

# 10.2 Importancias por Permutación (sobre columnas ORIGINALES)
r = permutation_importance(
    winner_pipe,            # pipeline completa
    X_test, y_test,
    n_repeats=10,
    random_state=RANDOM_STATE,
    scoring="neg_root_mean_squared_error"
)

feat_names = X_test.columns  # <-- CLAVE: mismos nombres que el X de entrada
imp = (pd.DataFrame({
        "feature": feat_names,
        "importance": r.importances_mean,
        "std": r.importances_std
     })
     .sort_values("importance", ascending=False)
     .head(15)
)
print("\nTop-15 importancias (perm, columnas originales):")
print(imp.to_string(index=False))

# 10.3 Errores: resumen + peores casos
y_hat = winner_pipe.predict(X_test)
y_pp  = postprocess_preds(y_hat, POLICY)
res   = pd.DataFrame({
    "y_true": y_test.reset_index(drop=True),
    "y_pred": pd.Series(y_pp)
})
res["abs_err"] = (res["y_true"] - res["y_pred"]).abs()
print("\nResumen de |error|:")
print(res["abs_err"].describe(percentiles=[.1,.25,.5,.75,.9]).to_string())

print("\nPeores 10 casos (|error| alto):")
top_bad_idx = res["abs_err"].nlargest(10).index
print(pd.concat([res.loc[top_bad_idx], X_test.reset_index(drop=True).loc[top_bad_idx]], axis=1)
      .to_string(index=False))

# 10.4 Métricas por subgrupos (ej. clase_salario)
if "clase_salario" in X_test.columns:
    by_cls = (pd.concat([X_test.reset_index(drop=True)[["clase_salario"]], res], axis=1)
              .groupby("clase_salario")["abs_err"]
              .agg(["count","mean","median"]))
    print("\nMAE por clase_salario:")
    print(by_cls.to_string())


[Policy] clipped_low: 0.000% | clipped_high: 0.000%

Top-15 importancias (perm, columnas originales):
         feature   importance        std
      smoker_yes 11001.294213 495.172353
             bmi  3240.656661 254.882929
             age  2547.839869 158.575499
        children   207.444110  66.041193
region_southwest     7.708388   5.929926
region_southeast     4.997960   3.239566
region_northwest     3.494475   4.086974
        sex_male    -2.065253   5.029555

Resumen de |error|:
count      268.000000
mean      2470.549641
std       3574.037408
min          6.906871
10%        489.538277
25%        925.408559
50%       1517.673893
75%       2305.828097
90%       3599.116406
max      21269.765057

Peores 10 casos (|error| alto):
     y_true       y_pred      abs_err  age    bmi  children  sex_male  smoker_yes  region_northwest  region_southeast  region_southwest
28476.73499  7206.969933 21269.765057   40 41.420         1     False       False              True             False  

In [31]:
Xtr = winner_pipe.named_steps["prep"].transform(X_test)
model = winner_pipe.named_steps["model"]
r2 = permutation_importance(model, Xtr, y_test, n_repeats=10,
                            random_state=RANDOM_STATE,
                            scoring="neg_root_mean_squared_error")
feat_names_ohe = winner_pipe.named_steps["prep"].get_feature_names_out()
imp_ohe = pd.DataFrame({"feature": feat_names_ohe,
                        "importance": r2.importances_mean,
                        "std": r2.importances_std}).sort_values("importance", ascending=False).head(20)


In [32]:
imp_ohe

,feature,importance,std
4,num__smoker_yes,11001.294213,495.172353
1,num__bmi,3240.656661,254.882929
0,num__age,2547.839869,158.575499
2,num__children,207.444110,66.041193
7,num__region_southwest,7.708388,5.929926
6,num__region_southeast,4.997960,3.239566
5,num__region_northwest,3.494475,4.086974
3,num__sex_male,-2.065253,5.029555
